In [1]:
import pandas as pd
import re
import numpy as np
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from collections import defaultdict, Counter
import math


In [2]:

def remove_punct_and_non_ascii(text):
    text = re.sub(r'[^\w\s]', '', text)
    return text.encode('ascii', 'ignore').decode('ascii')

def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'\d+', '<NUM>', text)
    return text

def tokenize(text):
    return word_tokenize(text)

def remove_stopwords(tokens):
    stop_words = set(stopwords.words('english'))
    return [t for t in tokens if t not in stop_words]

def stem_tokens(tokens):
    stemmer = PorterStemmer()
    return [stemmer.stem(t) for t in tokens]

def preprocess_pipeline(text):
    text = remove_punct_and_non_ascii(text)
    text = preprocess_text(text)
    tokens = tokenize(text)
    tokens = remove_stopwords(tokens)
    tokens = stem_tokens(tokens)
    return tokens

train_df = pd.read_csv('data/train.csv')
test_df = pd.read_csv('data/test.csv')

validation_df = train_df.sample(n=100, random_state=42)
new_train_df = train_df.drop(validation_df.index)

In [3]:
def preprocess_all(df):
    return [preprocess_pipeline(text) for text in df['text']]

new_train_tokens = preprocess_all(new_train_df)
validation_tokens = preprocess_all(validation_df)
test_tokens = preprocess_all(test_df)

min_count = max(1, int(len(new_train_df) * 0.01))

article_counts_uni = defaultdict(int)
for tokens in new_train_tokens:
    unique_tokens = set(tokens)
    for token in unique_tokens:
        article_counts_uni[token] += 1

unigram_vocab = {token for token, cnt in article_counts_uni.items() if cnt >= min_count}

def replace_oov(tokens, vocab):
    return [t if t in vocab else '<UNK>' for t in tokens]

new_train_tokens = [replace_oov(t, unigram_vocab) for t in new_train_tokens]
validation_tokens = [replace_oov(t, unigram_vocab) for t in validation_tokens]
test_tokens = [replace_oov(t, unigram_vocab) for t in test_tokens]

article_counts_bi = defaultdict(int)
for tokens in new_train_tokens:
    bigrams = set(zip(tokens[:-1], tokens[1:]))
    for bg in bigrams:
        article_counts_bi[bg] += 1

bigram_vocab = {bg for bg, cnt in article_counts_bi.items() if cnt >= min_count}

article_counts_tri = defaultdict(int)
for tokens in new_train_tokens:
    trigrams = set(zip(tokens[:-2], tokens[1:-1], tokens[2:]))
    for tg in trigrams:
        article_counts_tri[tg] += 1

trigram_vocab = {tg for tg, cnt in article_counts_tri.items() if cnt >= min_count}

V = len(unigram_vocab)
k = 1


In [4]:

# Unigram model
unigram_counts = Counter()
for tokens in new_train_tokens:
    unigram_counts.update(tokens)

N_uni = sum(unigram_counts.values())
unigram_probs = {token: (unigram_counts[token] + k) / (N_uni + k * V) for token in unigram_vocab}

# Bigram model
bigram_counts = Counter()
for tokens in new_train_tokens:
    bigrams = list(zip(tokens[:-1], tokens[1:]))
    bigram_counts.update(bigrams)

bigram_probs = {}
for bg in bigram_vocab:
    prev, curr = bg
    count_bg = bigram_counts[bg]
    count_prev = unigram_counts.get(prev, 0)
    bigram_probs[bg] = (count_bg + k) / (count_prev + k * V)

# Trigram model
trigram_counts = Counter()
for tokens in new_train_tokens:
    trigrams = list(zip(tokens[:-2], tokens[1:-1], tokens[2:]))
    trigram_counts.update(trigrams)

bigram_context_counts = Counter()
for tokens in new_train_tokens:
    bigrams_ctx = list(zip(tokens[:-1], tokens[1:]))
    bigram_context_counts.update(bigrams_ctx)

trigram_probs = {}
for tg in trigram_vocab:
    prev_prev, prev, curr = tg
    ctx = (prev_prev, prev)
    count_tg = trigram_counts[tg]
    count_ctx = bigram_context_counts.get(ctx, 0)
    trigram_probs[tg] = (count_tg + k) / (count_ctx + k * V)

In [5]:
# Evaluate perplexity
def calculate_perplexity(model_probs, tokens, model_type):
    log_sum = 0
    T = 0
    if model_type == 'unigram':
        for t in tokens:
            prob = unigram_probs.get(t, 1 / (N_uni + V))
            log_sum += math.log(prob)
        T = len(tokens)
    elif model_type == 'bigram':
        for i in range(1, len(tokens)):
            bg = (tokens[i-1], tokens[i])
            prob = bigram_probs.get(bg, 1 / (unigram_counts.get(tokens[i-1], 0) + V))
            log_sum += math.log(prob)
        T = len(tokens) - 1
    elif model_type == 'trigram':
        for i in range(2, len(tokens)):
            tg = (tokens[i-2], tokens[i-1], tokens[i])
            prob = trigram_probs.get(tg, 1 / (bigram_context_counts.get((tokens[i-2], tokens[i-1]), 0) + V))
            log_sum += math.log(prob)
        T = len(tokens) - 2
    else:
        raise ValueError("Invalid model type")
    return math.exp(-log_sum / T) if T > 0 else float('inf')

def evaluate_perplexity(model_type):
    perplexities = []
    for tokens in test_tokens:
        perplexities.append(calculate_perplexity(None, tokens, model_type))
    return np.mean(perplexities)

In [6]:
uni_ppl = evaluate_perplexity('unigram')
bi_ppl = evaluate_perplexity('bigram')
tri_ppl = evaluate_perplexity('trigram')

print(f"Unigram Perplexity: {uni_ppl}")
print(f"Bigram Perplexity: {bi_ppl}")
print(f"Trigram Perplexity: {tri_ppl}")

Unigram Perplexity: 4289.919588189338
Bigram Perplexity: 683.1776678821734
Trigram Perplexity: 1551.98516427819


In [7]:
# Interpolation model
def interpolated_prob(tokens, lambdas):
    l1, l2, l3 = lambdas
    total_log = 0
    for i in range(len(tokens)):
        uni_prob = unigram_probs.get(tokens[i], 1 / (N_uni + V))
        bg_prob = bigram_probs.get((tokens[i-1], tokens[i]), uni_prob) if i > 0 else uni_prob
        tg_prob = trigram_probs.get((tokens[i-2], tokens[i-1], tokens[i]), bg_prob) if i > 1 else bg_prob
        prob = l1 * tg_prob + l2 * bg_prob + l3 * uni_prob
        total_log += math.log(prob)
    return math.exp(-total_log / len(tokens))

best_ppl = float('inf')
best_lambdas = (0.4, 0.3, 0.3)

for l1 in np.linspace(0, 1, 11):
    for l2 in np.linspace(0, 1 - l1, 11):
        l3 = 1 - l1 - l2
        if l3 < 0:
            continue
        avg_ppl = np.mean([interpolated_prob(t, (l1, l2, l3)) for t in validation_tokens])
        if avg_ppl < best_ppl:
            best_ppl = avg_ppl
            best_lambdas = (l1, l2, l3)

interp_ppl = np.mean([interpolated_prob(t, best_lambdas) for t in test_tokens])

print(f"Best Lambdas: {best_lambdas}")
print(f"Interpolated Perplexity: {interp_ppl}")

Best Lambdas: (np.float64(0.0), np.float64(1.0), np.float64(0.0))
Interpolated Perplexity: 381.1258730408566
